In [29]:
import cv2
import numpy as np
import colorsys

from cv2 import VideoWriter, VideoWriter_fourcc

In [30]:
WIDTH = 512
HEIGHT = 512
FPS = 60
RADIUS = 2
STEP_SIZE = 3
STEPS = 100000

# define starting values
# normally hsv max is (360 deg, 100%, 100%) but because of the colorsys function
# all values are scaled between 0 and 1. Since S and V don't change and we're only
# taking a subset of color space we'll just leave those at their max but since
# we're exploring the hue space with the random walk we'll let hue move between
# 0 and 360 deg (with modular division) then scale it down before throwing it
# through the conversion function
hue = 0
SATURATION, VALUE = 1, 1
x, y = 255, 255

fourcc = VideoWriter_fourcc(*'MP42')
video = VideoWriter(
    f'video.avi',
    fourcc,
    float(FPS),
    (WIDTH, HEIGHT)
)

# draw the frame outside the loop so previous draws are persistant
# if you just want to animate a ball moving around you can move this within
# the for loop 
frame = np.zeros((WIDTH, HEIGHT, 3), dtype=np.uint8)

for i in range(100000):
    scaled_hue = hue / 360  #scale down hue
    rgb = colorsys.hsv_to_rgb(scaled_hue, SATURATION, VALUE)  # convert to rgb
    bgr = (rgb[2] * 255, rgb[1] * 255, rgb[0] * 255)  # cv2 only takes bgr so rearrange *255 to rescale
    center = (x, y)
    
    #draw circle
    cv2.circle(frame, center, RADIUS, bgr, -1)
    
    # walk through color space
    if np.random.randint(2) == 1:
        hue += STEP_SIZE
        hue %= 360
    else:
        hue -= STEP_SIZE
        hue %= 360
        
    # walk through space
    if np.random.randint(2) == 1:
        if np.random.randint(2) == 1:
            x += STEP_SIZE
            x %= WIDTH
        else:
            y += STEP_SIZE
            y %= HEIGHT
    else:
        if np.random.randint(2) == 1:
            x -= STEP_SIZE
            x %= WIDTH
        else:
            y -= STEP_SIZE
            y %= HEIGHT
    
    # Write frame
    video.write(frame) 
    # Display the resulting frame    
    cv2.imshow('frame',frame)
 
    # Press Q on keyboard to stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
video.release()
cv2.destroyAllWindows()